
# Advanced Time Series Forecasting — Data Generation & Baseline Modeling

**Contents**
- Generate synthetic hierarchical daily sales data (5 regions × 20 stores × 3 years)
- Fit baseline ETS (Holt–Winters) models per store
- Produce bottom-up aggregates and region/total ETS forecasts
- Reconcile forecasts (bottom-up + aggregate averaging)
- Compute basic evaluation metrics (MASE, wMAPE)
- Save outputs (CSV) for submission

This notebook is prepared for the project submission. Run the cells sequentially.


In [ ]:

# Imports
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)
os.makedirs("project_outputs", exist_ok=True)


In [ ]:

# Settings
n_regions = 5
stores_per_region = 20
freq = "D"           # daily data
years = 3
periods = years * 365  # approx daily
start_date = "2018-01-01"
dates = pd.date_range(start=start_date, periods=periods, freq=freq)

regions = [f"Region_{i+1}" for i in range(n_regions)]
stores = []
for r in regions:
    for s in range(1, stores_per_region+1):
        stores.append(f"{r}_Store_{s}")

# Seasonal patterns
weekly_season = 10 + 5 * np.sin(2 * np.pi * (np.arange(periods) % 7) / 7)
annual_season = 20 * np.sin(2 * np.pi * np.arange(periods) / 365.25)

rows = []
for i, store in enumerate(stores):
    region = store.split("_Store_")[0]
    base = np.random.uniform(50, 300) + (i % 10) * 2
    trend = np.random.uniform(-0.02, 0.1)
    noise_scale = np.random.uniform(5, 30)
    promo = np.zeros(periods)
    for _ in range(np.random.poisson(3)):
        start = np.random.randint(0, periods-30)
        length = np.random.randint(3, 14)
        promo[start:start+length] += np.random.uniform(20, 80)
    series = base + trend * np.arange(periods) + weekly_season + annual_season + promo + np.random.normal(0, noise_scale, periods)
    series = np.maximum(0, series).round(2)
    df = pd.DataFrame({"date": dates, "region": region, "store": store, "sales": series})
    rows.append(df)

data = pd.concat(rows, ignore_index=True)
data = data.sort_values(["store", "date"]).reset_index(drop=True)
data.to_csv("project_outputs/generated_sales_long.csv", index=False)
pivot = data.pivot(index="date", columns="store", values="sales")
pivot.to_csv("project_outputs/generated_sales_wide.csv", index=True)
print('Generated dataset saved to project_outputs/ (generated_sales_long.csv and generated_sales_wide.csv)')
display(data.head())

In [ ]:

# ETS baseline per store (Holt-Winters with weekly seasonality)
from tqdm import tqdm
horizon = 90
store_forecasts = {}
store_residuals = {}
store_pi = {}
for store in tqdm(pivot.columns):
    series = pivot[store].astype(float)
    try:
        model = ExponentialSmoothing(series, trend="add", seasonal="add", seasonal_periods=7)
        fitted = model.fit(optimized=True, use_boxcox=False, remove_bias=False)
        forecast = fitted.forecast(horizon)
        resid = fitted.resid.dropna()
        se = resid.std(ddof=1) if len(resid)>1 else np.std(series)
        z = 1.6448536269514722
        lower = forecast - z * se
        upper = forecast + z * se
        store_forecasts[store] = forecast.values
        store_residuals[store] = resid.values
        store_pi[store] = np.vstack([lower, upper]).T
    except Exception as e:
        last = series.iloc[-1]
        store_forecasts[store] = np.repeat(last, horizon)
        store_residuals[store] = np.zeros(max(1, len(series)-1))
        store_pi[store] = np.vstack([np.repeat(last*0.8, horizon), np.repeat(last*1.2, horizon)]).T

forecast_dates = pd.date_range(start=pivot.index[-1] + pd.Timedelta(days=1), periods=horizon, freq='D')
forecast_df = pd.DataFrame(index=forecast_dates, columns=pivot.columns)
for store in pivot.columns:
    forecast_df[store] = store_forecasts[store]

forecast_df.to_csv('project_outputs/bottom_level_store_forecasts.csv', index=True)
print('Saved bottom-level store forecasts to project_outputs/bottom_level_store_forecasts.csv')

In [ ]:

# Aggregate bottom-up to regions and total
regions = [f"Region_{i+1}" for i in range(n_regions)]
region_fc_bottom = {r: forecast_df[[c for c in forecast_df.columns if c.startswith(r + "_Store_")]].sum(axis=1) for r in regions}
total_fc_bottom = pd.DataFrame(region_fc_bottom, index=forecast_df.index).sum(axis=1)

# Fit ETS at region & total levels (aggregated historical)
region_hist = {}
for r in regions:
    cols = [c for c in pivot.columns if c.startswith(r + "_Store_")]
    region_hist[r] = pivot[cols].sum(axis=1)
region_hist_df = pd.DataFrame(region_hist, index=pivot.index)
total_series = region_hist_df.sum(axis=1)

region_ets_fc = {}
region_ets_pi = {}
for r in regions:
    series = region_hist_df[r].astype(float)
    try:
        fitted = ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=7).fit(optimized=True)
        fc = fitted.forecast(horizon)
        resid = fitted.resid.dropna()
        se = resid.std(ddof=1) if len(resid)>1 else np.std(series)
        z = 1.6448536269514722
        region_ets_fc[r] = fc.values
        region_ets_pi[r] = np.vstack([fc - z*se, fc + z*se]).T
    except:
        region_ets_fc[r] = np.repeat(series.iloc[-1], horizon)
        region_ets_pi[r] = np.vstack([np.repeat(series.iloc[-1]*0.8, horizon), np.repeat(series.iloc[-1]*1.2, horizon)]).T

# Total ETS
try:
    fitted_total = ExponentialSmoothing(total_series, trend='add', seasonal='add', seasonal_periods=7).fit(optimized=True)
    total_fc_ets = fitted_total.forecast(horizon)
    total_resid = fitted_total.resid.dropna()
    total_se = total_resid.std(ddof=1) if len(total_resid)>1 else np.std(total_series)
    total_pi = np.vstack([total_fc_ets - 1.6448536269514722*total_se, total_fc_ets + 1.6448536269514722*total_se]).T
except:
    total_fc_ets = np.repeat(total_series.iloc[-1], horizon)
    total_pi = np.vstack([np.repeat(total_series.iloc[-1]*0.8, horizon), np.repeat(total_series.iloc[-1]*1.2, horizon)]).T

region_fc_bottom_df = pd.DataFrame(region_fc_bottom, index=forecast_df.index)
region_ets_fc_df = pd.DataFrame(region_ets_fc, index=forecast_df.index)


In [ ]:

# Reconcile by averaging bottom-up and aggregate ETS forecasts (50-50)
reconciled_region = 0.5 * region_fc_bottom_df + 0.5 * region_ets_fc_df
reconciled_total = 0.5 * total_fc_bottom + 0.5 * total_fc_ets.values

top_level_out = pd.DataFrame(index=forecast_df.index)
top_level_out['Total'] = reconciled_total.values
for r in regions:
    top_level_out[r] = reconciled_region[r].values

# Attach 90% PIs based on ETS residual std proxies
z = 1.6448536269514722
for r in regions:
    se = np.std(region_hist_df[r].diff().dropna()) if len(region_hist_df[r])>2 else 1.0
    top_level_out[r + '_lower90'] = top_level_out[r] - z * se
    top_level_out[r + '_upper90'] = top_level_out[r] + z * se

se_total = np.std(total_series.diff().dropna()) if len(total_series)>2 else 1.0
top_level_out['Total_lower90'] = top_level_out['Total'] - z * se_total
top_level_out['Total_upper90'] = top_level_out['Total'] + z * se_total

top_level_out.to_csv('project_outputs/top_level_forecasts_reconciled.csv', index=True)
print('Saved reconciled top-level forecasts to project_outputs/top_level_forecasts_reconciled.csv')
display(top_level_out.head())

In [ ]:

# Evaluation: simple holdout-based MASE and wMAPE at region/total levels
holdout = 90
true_region_hold = region_hist_df.iloc[-holdout:]
true_total_hold = total_series.iloc[-holdout:]

# naive forecast: last observed repeated
naive_fc = pd.DataFrame(np.tile(pivot.iloc[-holdout-1].values, (holdout, 1)), columns=pivot.columns, index=true_region_hold.index)
naive_region_hold = {r: naive_fc[[c for c in pivot.columns if c.startswith(r + '_Store_')]].sum(axis=1) for r in regions}
naive_region_hold_df = pd.DataFrame(naive_region_hold, index=true_region_hold.index)
naive_total_hold = naive_region_hold_df.sum(axis=1)

def mase(insample, naive_forecast, actual):
    scale = np.mean(np.abs(np.diff(insample)))
    if scale == 0:
        scale = 1.0
    return np.mean(np.abs(actual - naive_forecast)) / scale

def wmape(actual, forecast):
    num = np.sum(np.abs(actual - forecast))
    den = np.sum(np.abs(actual))
    return num / den if den != 0 else np.nan

metrics = {}
for r in regions:
    insample = region_hist_df[r].iloc[:-holdout].values
    actual = true_region_hold[r].values
    forecast_vals = naive_region_hold_df[r].values
    metrics[r] = {'MASE': mase(insample, forecast_vals, actual), 'wMAPE': wmape(actual, forecast_vals)}

insample_total = total_series.iloc[:-holdout].values
metrics['Total'] = {'MASE': mase(insample_total, naive_total_hold.values, true_total_hold.values), 'wMAPE': wmape(true_total_hold.values, naive_total_hold.values)}

metrics_df = pd.DataFrame(metrics).T
display(metrics_df)
metrics_df.to_csv('project_outputs/evaluation_metrics.csv', index=True)
print('Saved evaluation metrics to project_outputs/evaluation_metrics.csv')

In [ ]:

# Save a short report and README
report_text = f"""Project: Advanced Time Series Forecasting with Hierarchical Probabilistic Modeling (Generated)
- Data: Simulated daily sales for {n_regions} regions x {stores_per_region} stores over {years} years.
- Models used (baseline): ETS (Holt-Winters) per store with weekly seasonality. Forecast horizon: {horizon} days.
- Reconciliation strategies implemented: Bottom-up (summing store forecasts) and aggregate ETS averaging (50-50).
- Evaluation: MASE and wMAPE computed on a {holdout}-day holdout for region and top-level aggregates.
- Outputs saved in project_outputs/ folder.
"""

with open('project_outputs/project_report.txt', 'w') as f:
    f.write(report_text)

readme = """# advanced-hpf-forecasting

This repository contains code and outputs for the Advanced Time Series Forecasting project.

Run the notebook `data_generation_and_modeling.ipynb` to reproduce the dataset, baseline models, reconciled forecasts, and evaluation metrics.

Files in `project_outputs/` are ready for submission.
"""
with open('project_outputs/README.md', 'w') as f:
    f.write(readme)
print('Saved project_report.txt and README.md in project_outputs/')